In [1]:
from nnunet.evaluation.metrics import ConfusionMatrix, ALL_METRICS
import nibabel
import numpy as np
import os
from batchgenerators.utilities.file_and_folder_operations import save_json, subfiles, join
from tqdm import tqdm
import surface_distance as surfdist
import prettytable as pt
from skimage.morphology import skeletonize, skeletonize_3d

METRICS =[
    ['Accuracy','Precision','Recall'], # 体素精度，准确度(TP + TN) / (TP + FP + FN + TN)，精确度 TP / (TP + FP)，召回率(TP / (TP + FN))
    ['Dice','clDice'], # 相似度，骰子系数和中心线系数
    ['Hausdorff Distance 95','Avg. Symmetric Surface Distance'] # 距离精度,hd95,assd
] 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [2]:
def cl_score(v, s):
    """[this function computes the skeleton volume overlap]
    Args:
        v ([bool]): [image]
        s ([bool]): [skeleton]
    Returns:
        [float]: [computed skeleton volume intersection]
    """
    return np.sum(v*s)/np.sum(s)


def clDice(v_p, v_l):
    """[this function computes the cldice metric]
    Args:
        v_p ([bool]): [predicted image]
        v_l ([bool]): [ground truth image]
    Returns:
        [float]: [cldice metric]
    """
    if len(v_p.shape)==2:
        tprec = cl_score(v_p,skeletonize(v_l))
        tsens = cl_score(v_l,skeletonize(v_p))
    elif len(v_p.shape)==3:
        tprec = cl_score(v_p,skeletonize_3d(v_l))
        tsens = cl_score(v_l,skeletonize_3d(v_p))
    return 2*tprec*tsens/(tprec+tsens)

def get_files_metrics(matrix,result=None):
    surface_distances = 0
    for item in result:
        if result[item]==None:
            result[item]=[]
                
        if item =='clDice':
            result[item].append(clDice(matrix['data'].test,matrix['data'].reference))
            continue

        if item =='HausdorffDistance 95':
            if surface_distances==0:
                surface_distances = surfdist.compute_surface_distances(matrix['data'].test==1, matrix['data'].reference==1, spacing_mm=matrix['voxel_spacing'])
            hd95 = surfdist.compute_robust_hausdorff(surface_distances, 95)
            result[item].append(hd95)
            continue

        if item =='Avg. Symmetric Surface Distance':
            if surface_distances==0:
                surface_distances = surfdist.compute_surface_distances(matrix['data'].test==1, matrix['data'].reference==1, spacing_mm=matrix['voxel_spacing'])                    
            asd = surfdist.compute_average_surface_distance(surface_distances)
            assd = (asd[0]+asd[1])/2
            result[item].append(assd)
            continue
        
        result[item].append(ALL_METRICS[item](confusion_matrix=matrix['data']))
    return result

def get_fold_result(seg_fold,label_fold,cut):
    patients = os.listdir(seg_fold)
    result = {}.fromkeys([item for sublist in METRICS for item in sublist])
    matrix = []
    for i in tqdm(patients):
        try:
            seg = nibabel.load(join(seg_fold,i)).get_fdata().astype(np.uint8)
            label = nibabel.load(join(label_fold,i))
            voxel_spacing = label.header['pixdim'][1:4]
            label = label.get_fdata().astype(np.uint8)
            if cut:
                zz,_,_ = np.where(label)
                seg = seg[np.min(zz): np.max(zz)+1]
                label = label[np.min(zz): np.max(zz)+1]
            matrix={'data':ConfusionMatrix(test=seg==1,reference=label==1),'voxel_spacing':voxel_spacing}
            result = get_files_metrics(matrix,result)
        except:
            # print('missing '+i)
            continue
    return result

def evaluate(seg_dir,label_dir,fold_list=[0],cut=True):
    tb = pt.PrettyTable()
    tb.field_names = ['Fold Name']+[item for sublist in METRICS for item in sublist]
    results = {i:{} for i in fold_list}
    average_result_fold = []
    for fold in fold_list:
        result = get_fold_result(join(seg_dir,'fold'+str(fold)),label_dir,cut)
        result_list = []
        item = []
        for m in result:
            nums = result[m]
            result[m] = {'data':nums}
            nums = np.array(nums)
            result[m]['mean']=nums.mean()
            result[m]['std']=nums.std()
            result_list.append('{:.5f}±{:.5f}'.format(nums.mean(),nums.std()))
            item.append(nums.mean())
        tb.add_row(['fold'+str(fold)]+result_list)
        average_result_fold.append(item)
        results[fold]=result
        del result

    average_result_fold = np.stack(average_result_fold)
    avg_mean = average_result_fold.mean(axis=0)
    avg_std = average_result_fold.std(axis=0)
    average_result_fold = [ '{:.5f}±{:.5f}'.format(avg_mean[i],avg_std[i]) for i in range(len(tb.field_names)-1)]
    tb.add_row(['fold avg']+average_result_fold) 

    return results,tb

### nnUnet原版结果

In [22]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 12/12 [02:26<00:00, 12.25s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold0   | 0.99958±0.00012 | 0.91569±0.03634 | 0.89025±0.06245 | 0.90068±0.03029 | 0.92069±0.01767 |    1.52022±1.36220    |         0.50424±0.28079         |
|   fold1   | 0.99957±0.00016 | 0.88732±0.05572 | 0.90750±0.05763 | 0.89484±0.03421 | 0.91059±0.03774 |    1.61739±1.06492    |         0.45074±0.18585         |
|   fold2   | 0.99959±0.00012 | 0.93036±0.03688 | 0.84848±0.05414 | 0.88587±0.02935 | 0.88663±0.04400 |    2.76190±1.92490    |         0.70309±0.38811         |
|   fold3   | 0.99959±0.0001

### DCRE

In [14]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_DCRE'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 12/12 [02:46<00:00, 13.88s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold0   | 0.99959±0.00011 | 0.91362±0.03506 | 0.89696±0.05646 | 0.90346±0.02746 | 0.92972±0.01104 |    1.40237±1.09546    |         0.41743±0.17220         |
|   fold1   | 0.99960±0.00015 | 0.89266±0.05262 | 0.91616±0.05414 | 0.90209±0.03202 | 0.92603±0.03404 |    1.23570±0.55351    |         0.35103±0.13700         |
|   fold2   | 0.99961±0.00010 | 0.93770±0.03551 | 0.85428±0.04693 | 0.89254±0.02270 | 0.90545±0.04121 |    1.82587±1.04062    |         0.55981±0.38634         |
|   fold3   | 0.99960±0.0001

### Vnet

In [16]:
seg_dir = r'/root/workspace/data/train_models/Task050/Vnet/'
label_dir = r'/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 12/12 [03:01<00:00, 15.14s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold0   | 0.99933±0.00020 | 0.81920±0.05910 | 0.89588±0.05203 | 0.85306±0.03062 | 0.80603±0.05380 |   15.98336±12.08971   |         2.46200±1.17453         |
|   fold1   | 0.99940±0.00025 | 0.82160±0.06716 | 0.90572±0.04242 | 0.85973±0.04287 | 0.79810±0.06318 |    12.21216±7.60595   |         2.05579±0.69323         |
|   fold2   | 0.99937±0.00028 | 0.84104±0.09689 | 0.84702±0.05730 | 0.83915±0.05654 | 0.77603±0.08743 |    13.74628±7.69293   |         2.20678±0.79836         |
|   fold3   | 0.99934±0.0002

### MedNet

In [17]:
seg_dir = r'/root/workspace/data/train_models/Task050/MedNet/'
label_dir = r'/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:37<00:00, 12.12s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold0   | 0.99939±0.00014 | 0.89155±0.03824 | 0.82139±0.06149 | 0.85298±0.03372 | 0.81664±0.03658 |    10.07663±9.95358   |         1.61525±0.86042         |
|   fold1   | 0.99944±0.00017 | 0.88263±0.05044 | 0.83460±0.06420 | 0.85543±0.03842 | 0.80170±0.05982 |    7.59341±4.89833    |         1.18975±0.43822         |
|   fold2   | 0.99943±0.00013 | 0.91400±0.04759 | 0.77660±0.04523 | 0.83797±0.02793 | 0.79538±0.06572 |    10.02737±5.23543   |         1.53677±0.49541         |
|   fold3   | 0.99941±0.0001


---

# V4

【下采样层数为3】

比原本性能变差,也就是说减小感受野后结果下降了，召回、Dice、clDice、HD95 ASSD都变差

In [26]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v4/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:49<00:00, 13.03s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold0   | 0.99952±0.00011 | 0.89276±0.04222 | 0.88554±0.05624 | 0.88719±0.02815 | 0.88390±0.02450 |    3.76400±4.10576    |         1.03074±0.47363         |
|   fold2   | 0.99959±0.00010 | 0.93935±0.03359 | 0.84219±0.04341 | 0.88693±0.02370 | 0.88034±0.05014 |    3.54446±3.76429    |         0.79890±0.54403         |
|  fold avg | 0.99956±0.00004 | 0.91606±0.02329 | 0.86387±0.02168 | 0.88706±0.00013 | 0.88212±0.00178 |    3.65423±0.10977    |         0.91482±0.11592         |
+-----------+---------------

## V3——e200
【下采样5 开启ccp centerline tcl，分支有三层，融合两层】

虽然比原本的数据提高了性能，但是各项数据提升并不是很大，也没有dcre提升高很多，并没有太大优势
这里的epoch是200

In [20]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v3/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:41<00:00, 12.44s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99961±0.00011 | 0.94782±0.03148 | 0.84468±0.04892 | 0.89183±0.02495 | 0.89280±0.04632 |    2.33812±1.78881    |         0.60187±0.42031         |
|  fold avg | 0.99961±0.00000 | 0.94782±0.00000 | 0.84468±0.00000 | 0.89183±0.00000 | 0.89280±0.00000 |    2.33812±0.00000    |         0.60187±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


---
此处为了更快实验，将epoch改为100来进行实验
# baseline 

epoch100

In [4]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_baseline/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:33<00:00, 11.83s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99957±0.00013 | 0.91799±0.05100 | 0.85432±0.04735 | 0.88328±0.03199 | 0.86496±0.07202 |    3.65753±2.73053    |         0.89059±0.43443         |
|  fold avg | 0.99957±0.00000 | 0.91799±0.00000 | 0.85432±0.00000 | 0.88328±0.00000 | 0.86496±0.00000 |    3.65753±0.00000    |         0.89059±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# dcre 
虽然很简单，但是很有效，离谱？
所有指标都提高了，再敏感性维持的情况下，提高了两个点的查准，同时clDice大幅提升，Dice提高接近一个点，HD95和ASSD都减小明显

In [4]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_DC_e100/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:11<00:00, 10.10s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99960±0.00011 | 0.93049±0.03460 | 0.85694±0.04611 | 0.89080±0.02360 | 0.89038±0.04497 |    2.68443±2.40703    |         0.70974±0.46261         |
|  fold avg | 0.99960±0.00000 | 0.93049±0.00000 | 0.85694±0.00000 | 0.89080±0.00000 | 0.89038±0.00000 |    2.68443±0.00000    |         0.70974±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


## V3_e100
【下采样5 开启ccp centerline tcl，分支有三层，融合两层 alpha=0.2】

epoch是100

体素级指标提升并不是很大，甚至recall都下降了，召回率下降就是敏感性下降了

所以Dice也下降了，但奇怪的是clDice和边界指标提升了，也就是说整体连通性和边界上结果更好了？但是整体的精度却并没有很好的提升

In [3]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v3_e100/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [03:13<00:00, 14.86s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99956±0.00012 | 0.91877±0.04111 | 0.84777±0.05383 | 0.88007±0.02996 | 0.88365±0.04817 |    3.05817±2.16926    |         0.77515±0.40832         |
|  fold avg | 0.99956±0.00000 | 0.91877±0.00000 | 0.84777±0.00000 | 0.88007±0.00000 | 0.88365±0.00000 |    3.05817±0.00000    |         0.77515±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


## V3_v50
【下采样5 开启ccp centerline tcl，分支有三层，融合两层 alpha=0.5】

epoch是100

【相比较于baseline】

体素级指标中敏感性其实降低了，但查准提高了接近两个点，其实在一般分类中，敏感性降低查准提高是符合常理的，但是这里是分割，我们并没有人为调整阈值，所以模型对于目标血管的概念更为清晰了，分割结果更准确、更可靠了。

不过虽然从整体像素分类精度上看，分类的敏感性降低了，整体相似度Dice结果却并没有变差，甚至反而高了一点点，

clDice提高的也非常多,接近2个点，这说明整体连通性是提升了的，或者说是有一些断掉的、不连续的、消失的部分被分割出来了，所以整体的连通性结果变好了

HD95和ASSD都下降比较多，ASSD下降16%，HD95下降了25%，这可以说明分割的边缘形状更完整了、更好了

【相比较于v3 alpha:0.2】

除了敏感性持平意外，其他结果都有了提升

In [5]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v3_50/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [03:10<00:00, 14.66s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99959±0.00011 | 0.93373±0.03442 | 0.84757±0.04455 | 0.88725±0.02385 | 0.88504±0.04663 |    2.72183±2.03815    |         0.74453±0.43098         |
|  fold avg | 0.99959±0.00000 | 0.93373±0.00000 | 0.84757±0.00000 | 0.88725±0.00000 | 0.88504±0.00000 |    2.72183±0.00000    |         0.74453±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


## V3_v70
【下采样5 开启ccp centerline tcl，分支有三层，融合两层 alpha=0.5】

recall倒是提高了，但是查准提高的并不多，Dice达到了最高，但是clDice并不是最高，hd95和assd并不是最低

In [5]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v3_70/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:14<00:00, 10.38s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99960±0.00011 | 0.92768±0.03660 | 0.85758±0.04501 | 0.88980±0.02290 | 0.88189±0.04631 |    2.71553±1.90417    |         0.72826±0.37954         |
|  fold avg | 0.99960±0.00000 | 0.92768±0.00000 | 0.85758±0.00000 | 0.88980±0.00000 | 0.88189±0.00000 |    2.71553±0.00000    |         0.72826±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


## V3_v80
【下采样5 开启ccp centerline tcl，分支有三层，融合两层 alpha=0.8】

epoch是100

【相比较于baseline】
除了recall都有提升

【相比较于v3 alpha:0.2】
其实边缘结果更好一些了，但是吧其他的提升幅度并不大，召回甚至更低了


In [6]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v3_80/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:35<00:00, 11.93s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99958±0.00012 | 0.93250±0.03670 | 0.84345±0.04987 | 0.88419±0.02708 | 0.88432±0.04642 |    2.55993±1.69216    |         0.70347±0.37572         |
|  fold avg | 0.99958±0.00000 | 0.93250±0.00000 | 0.84345±0.00000 | 0.88419±0.00000 | 0.88432±0.00000 |    2.55993±0.00000    |         0.70347±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# V4

**更早的分支是否效果是否会更好？**

【下采样5 开启ccp centerline tcl，分支有四层，融合三层 alpha=0.5】


In [4]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_V4/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:17<00:00, 10.57s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99958±0.00011 | 0.93310±0.03652 | 0.84201±0.04560 | 0.88380±0.02467 | 0.87781±0.05234 |    3.28377±2.44272    |         0.83201±0.46275         |
|  fold avg | 0.99958±0.00000 | 0.93310±0.00000 | 0.84201±0.00000 | 0.88380±0.00000 | 0.87781±0.00000 |    3.28377±0.00000    |         0.83201±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


## V5

差的离谱，下采样层变少之后结果变差了，可以看到感受野变小确实会影响结果

In [28]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v5/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:44<00:00, 12.69s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99956±0.00013 | 0.91710±0.04982 | 0.84795±0.05192 | 0.87922±0.03231 | 0.87301±0.05543 |    3.90310±3.11167    |         0.91509±0.55992         |
|  fold avg | 0.99956±0.00000 | 0.91710±0.00000 | 0.84795±0.00000 | 0.87922±0.00000 | 0.87301±0.00000 |    3.90310±0.00000    |         0.91509±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# V6

**上采样共享是否会影响结果？**

v3基础上修改的，此处上采样层不共享了

【下采样5 开启ccp centerline tcl，分支有三层，融合两层 alpha=0.5】

确实有变化，变化趋势也产生了变化，也就是说基于反卷积的上采样共享是会影响计算结果的，但是此处效果提升仍然不是很大

过程中发现tcl损失下降的很慢

但是recall有明显提高，但是查准却下降了

In [5]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_V6/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:16<00:00, 10.52s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99959±0.00011 | 0.92136±0.03758 | 0.86050±0.04685 | 0.88832±0.02351 | 0.88344±0.04824 |    2.75220±2.04338    |         0.82408±0.51641         |
|  fold avg | 0.99959±0.00000 | 0.92136±0.00000 | 0.86050±0.00000 | 0.88832±0.00000 | 0.88344±0.00000 |    2.75220±0.00000    |         0.82408±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# V6 70

**调整alpha参数，自身信息多会提升性能吗**

v3基础上修改的，此处上采样层不共享了

【下采样5 开启centerline tcl ccp，分支有三层，融合两层 alpha=0.7】

# V7

**CCP是否会影响tcl的结果？**

v3基础上修改的，此处上采样层不共享了，并且关闭了CCP，查看如果没有CCP的话结果会怎样

【下采样5 开启centerline tcl，分支有三层，融合两层 alpha=0.5】

**关掉ccp并不会带来效果提升**

In [8]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v7/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:20<00:00, 10.84s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99958±0.00011 | 0.92179±0.04106 | 0.85714±0.04703 | 0.88672±0.02605 | 0.88174±0.05121 |    2.83023±1.97647    |         0.71421±0.36363         |
|  fold avg | 0.99958±0.00000 | 0.92179±0.00000 | 0.85714±0.00000 | 0.88672±0.00000 | 0.88174±0.00000 |    2.83023±0.00000    |         0.71421±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# V8

**tcl损失的占比？**

v3基础上修改的，此处上采样层不共享了，

【下采样5 开启centerline ccp tcl，tcl损失系数为0.0001，分支有三层，融合两层 alpha=0.5】

In [11]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v8/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:16<00:00, 10.54s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99960±0.00011 | 0.92519±0.04003 | 0.86032±0.04612 | 0.89001±0.02451 | 0.87862±0.04907 |    3.00219±2.51399    |         0.80201±0.48341         |
|  fold avg | 0.99960±0.00000 | 0.92519±0.00000 | 0.86032±0.00000 | 0.89001±0.00000 | 0.87862±0.00000 |    3.00219±0.00000    |         0.80201±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# V8-1

**tcl损失的占比？**

v3基础上修改的，此处上采样层不共享了，

【下采样5 开启centerline ccp tcl，tcl损失系数为0.0001，分支有三层，融合两层 alpha=1】

效果变差啦

In [16]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v8_1/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:20<00:00, 10.79s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99959±0.00012 | 0.92927±0.04126 | 0.85206±0.04902 | 0.88739±0.02795 | 0.87372±0.05826 |    3.11999±2.44758    |         0.79655±0.52583         |
|  fold avg | 0.99959±0.00000 | 0.92927±0.00000 | 0.85206±0.00000 | 0.88739±0.00000 | 0.87372±0.00000 |    3.11999±0.00000    |         0.79655±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


In [22]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v8_0/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:37<00:00, 12.11s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99958±0.00012 | 0.92212±0.04199 | 0.85811±0.04794 | 0.88730±0.02620 | 0.88183±0.05219 |    3.00173±2.53843    |         0.78288±0.46514         |
|  fold avg | 0.99958±0.00000 | 0.92212±0.00000 | 0.85811±0.00000 | 0.88730±0.00000 | 0.88183±0.00000 |    3.00173±0.00000    |         0.78288±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


In [23]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v8_20/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:34<00:00, 11.91s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99959±0.00011 | 0.93038±0.03534 | 0.85399±0.04382 | 0.88925±0.02360 | 0.88101±0.05674 |    2.89829±2.31299    |         0.75032±0.44814         |
|  fold avg | 0.99959±0.00000 | 0.93038±0.00000 | 0.85399±0.00000 | 0.88925±0.00000 | 0.88101±0.00000 |    2.89829±0.00000    |         0.75032±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


In [24]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v8_70/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:33<00:00, 11.82s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99959±0.00011 | 0.92473±0.04065 | 0.85545±0.04538 | 0.88726±0.02547 | 0.87772±0.05298 |    3.12183±2.45034    |         0.81901±0.49550         |
|  fold avg | 0.99959±0.00000 | 0.92473±0.00000 | 0.85545±0.00000 | 0.88726±0.00000 | 0.87772±0.00000 |    3.12183±0.00000    |         0.81901±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# V9

**tcl损失的占比？**

v3基础上修改的，此处上采样层不共享了，

【下采样5 开启centerline ccp tcl，tcl损失系数为0.001，分支有三层，融合两层 alpha=0.5】

哦豁 效果变差了

看来对齐的太狠也不行

In [14]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/infer_v9/'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:12<00:00, 10.21s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99958±0.00013 | 0.92288±0.04881 | 0.85330±0.04825 | 0.88486±0.02971 | 0.87115±0.06126 |    3.42504±2.71153    |         0.85202±0.46273         |
|  fold avg | 0.99958±0.00000 | 0.92288±0.00000 | 0.85330±0.00000 | 0.88486±0.00000 | 0.87115±0.00000 |    3.42504±0.00000    |         0.85202±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


# HF 

顶层融合
alpha = 0.2

在顶层融合比在下面层级效果好很多

In [10]:
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/hf_v1'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:36<00:00, 12.01s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99958±0.00011 | 0.91417±0.04277 | 0.86460±0.04672 | 0.88695±0.02412 | 0.88807±0.04696 |    2.83609±2.38884    |         0.68876±0.42260         |
|  fold avg | 0.99958±0.00000 | 0.91417±0.00000 | 0.86460±0.00000 | 0.88695±0.00000 | 0.88807±0.00000 |    2.83609±0.00000    |         0.68876±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


GT 特征门控
alpha 0.2 tcl参数为0.001 numpool为4

HF v2

tcl参数调整为了0.001

HF v1 50

alpha参数调整为0.5

In [11]:
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/gf_v1'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:15<00:00, 10.43s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99957±0.00013 | 0.91768±0.05201 | 0.85414±0.05075 | 0.88269±0.03117 | 0.87884±0.05863 |    3.14658±2.14453    |         0.77216±0.38434         |
|  fold avg | 0.99957±0.00000 | 0.91768±0.00000 | 0.85414±0.00000 | 0.88269±0.00000 | 0.87884±0.00000 |    3.14658±0.00000    |         0.77216±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+


In [12]:
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/hf_v1_50'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)

100%|██████████| 13/13 [02:14<00:00, 10.31s/it]

+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy    |    Precision    |      Recall     |       Dice      |      clDice     | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
|   fold2   | 0.99959±0.00012 | 0.91779±0.04620 | 0.86402±0.04837 | 0.88830±0.02819 | 0.88094±0.05826 |    3.11748±2.48958    |         0.74269±0.45144         |
|  fold avg | 0.99959±0.00000 | 0.91779±0.00000 | 0.86402±0.00000 | 0.88830±0.00000 | 0.88094±0.00000 |    3.11748±0.00000    |         0.74269±0.00000         |
+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------------+---------------------------------+
